In [2]:
from pathlib import Path
import sys, inspect

PROJECT_ROOT = Path.cwd().parents[1]
sys.path.append(str(PROJECT_ROOT))
import dcf_models as dcf

print([n for n in dir(dcf) if not n.startswith("_")])
# If short, you can also peek:
# print(inspect.getsource(dcf))

# Try likely function names and alias them to the names you want to use:
dcf_npv = (
    getattr(dcf, "dcf_npv", None)
    or getattr(dcf, "npv", None)
    or getattr(dcf, "present_value", None)
    or getattr(dcf, "discounted_cash_flow", None)
)

['calculate_dcf']


In [4]:
# 1) Minimal NPV function (annual compounding; times in years)
def dcf_npv(cash_flows, r, times=None, freq=1):
    """
    cash_flows: list of CFs [CF1, CF2, ...]
    r: annual discount rate (decimal, e.g. 0.045)
    times: optional list of times in YEARS; default = 1..N
    freq: if CFs are per period (e.g., semiannual), set freq=2 and times in years
    """
    if times is None:
        times = range(1, len(cash_flows)+1)
    if freq == 1:
        return sum(cf / ((1 + r) ** t) for cf, t in zip(cash_flows, times))
    else:
        # per-period compounding when freq > 1
        return sum(cf / ((1 + r/freq) ** int(round(t*freq))) for cf, t in zip(cash_flows, times))

In [5]:
# 2) Example + save artifact
from pathlib import Path
import os

PROJECT_ROOT = Path.cwd().parents[1]
out_dir = PROJECT_ROOT / "outputs" / "04_DCF"
out_dir.mkdir(parents=True, exist_ok=True)

cash_flows = [10, 10, 10, 110]             # yearly coupon 10, redemption 100
npv = dcf_npv(cash_flows=cash_flows, r=0.045)

print(f"DCF NPV: {npv:.2f}")
with open(out_dir / "dcf_npv.txt", "w") as f:
    f.write(f"NPV={npv:.2f}")
print("✔ Saved:", out_dir / "dcf_npv.txt")

DCF NPV: 119.73
✔ Saved: /Users/katherinecohen/Documents/FixedIncomePortfolio/outputs/04_DCF/dcf_npv.txt


Valuation Decomposition
PV of coupons (4 payments of 10): 35.88
PV of principal (100 at 𝑡 = 4): 83.86
Total price: 119.73 (= 35.88 + 83.86)

Interest-Rate Risk (per 100 face)

Computed at 𝑦 = 4.50% with annual coupons/maturity T=4 years:
Macaulay duration: 3.534 years
Modified duration: 3.382 years
Convexity: 15.481 years²
DV01: 0.0405 (≈ 4.05 per bp)

At this price and yield, ±10 bp implies roughly ∓0.34–0.35% (before convexity), and convexity adds a slight cushion on sell-offs and a slight boost on rallies for larger moves.

Implications & Recommendations

Valuation: At a 4.50% required return, 119.73 is fair value. If the market price is below this level, the bond is undervalued (positive NPV at 4.5%); above it, overvalued versus your hurdle rate.

Risk Posture: With moderate duration ~3.38y, rate risk is moderate. For mandate-level duration targeting, hedge or add duration accordingly using the DV01 to scale.

Carry vs. price risk: This is a carry-rich position (10% coupon) whose premium will pull down toward par. In a bearish rates view, consider trimming exposure or overlaying a modest short-duration hedge to reduce steepening/parallel-shift sensitivity by the desired number of bp*DV01. In a bullish view, maintaining exposure captures convexity-aided upside.

Reinvestment & conventions: This result assumes annual compounding, no taxes/fees, and reinvestment at the discount rate. For production use, align frequency (e.g., semiannual), day-count, and accrued-interest handling with market conventions to avoid small but persistent biases.

Reproducibility (key figures)

Price: 119.7314

PV(coupons)/PV(principal): 35.8753 / 83.8561

Macaulay / Modified duration: 3.5341 / 3.3819

Convexity: 15.4805

DV01: 0.04049

Reprice @ 5.00% / 4.00%: 117.7298 / 121.7794

These diagnostics provide both a point-in-time valuation and a set of actionable levers (duration/DV01, convexity, horizon carry) to align the position with your rate outlook and risk budget.